In [72]:
import geopandas as gpd
import pyarrow
import pandas as pd
import os
import haversine as hs
import math
import numpy as np

In [41]:
R = 6371e3
def calculate_bearing(homeLatitude, homeLongitude, destinationLatitude, destinationLongitude):
    rlat1   =   homeLatitude * (math.pi/180) 
    rlat2   =   destinationLatitude * (math.pi/180) 
    rlon1   =   homeLongitude * (math.pi/180) 
    rlon2   =   destinationLongitude * (math.pi/180)
    dlat    =   (destinationLatitude - homeLatitude) * (math.pi/180)
    dlon    =   (destinationLongitude - homeLongitude) * (math.pi/180)
    # Formula for bearing
    y = math.sin(rlon2 - rlon1) * math.cos(rlat2)
    x = math.cos(rlat1) * math.sin(rlat2) - math.sin(rlat1) * math.cos(rlat2) * math.cos(rlon2 - rlon1)
    # Haversine formula to find distance
    a = (math.sin(dlat/2) * math.sin(dlat/2)) + (math.cos(rlat1) * math.cos(rlat2) * (math.sin(dlon/2) * math.sin(dlon/2)))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    # Distance in metres
    distance = R * c 
    # Bearing in radians
    bearing = math.atan2(y, x)
    bearingDegrees = bearing * (180/math.pi)

    return distance, bearingDegrees

In [5]:
target_2631 = pd.read_parquet('data/GPS.nosync/filter/Sonoma_2631_new', engine='pyarrow')

In [6]:
n = len(target_2631)

In [7]:
(target_2631.loc[2]['la_datetime'] - target_2631.loc[1]['la_datetime']) / pd.Timedelta('1 second')

10.486944444444445

In [8]:
t = (target_2631.loc[10002]['la_datetime'] - target_2631.loc[1]['la_datetime']) / pd.Timedelta('1 hour')
loc1=(target_2631.loc[10002]['latitude'],target_2631.loc[10002]['longitude'])
loc2=(target_2631.loc[1]['latitude'],target_2631.loc[1]['longitude'])
x = hs.haversine(loc2,loc1,unit=hs.Unit.MILES)
x / t

-4.072801694479074

In [84]:
start_index = 0
start_id = target_2631.loc[start_index]['caid']
start_time = target_2631.loc[start_index]['la_datetime']
end_time = target_2631.loc[start_index]['la_datetime']
#data = {'index': [-1], 'caid': [start_id], 'start_time' : [start_time], 'end_time' : [end_time], 'count' : [0], 'is_process' : [False], 'is_success' : [False], 'res' : ['LINESTRING()'], 'time' : [0], 'speed' : [0], 'direction' : [0] }  
df_new = pd.DataFrame(columns=['index', 'caid', 'start_time', 'end_time', 'count', 'is_process', 'is_success', 'res', 'time', 'distance', 'speed', 'direction'])
# Create DataFrame  
#df_new = pd.DataFrame(data)
print("start index: %d, caid: %s" %(0, start_id))
end = n
#end = 10000
count = 0
is_first = True
res = 'LINESTRING('
time_array = []
distance_array = []
speed_array = []
direction_array = []
prev_time = 0
prev_lat = 0
prev_lon = 0
for index in range(0, end):
    data_id = target_2631.loc[index]['caid']
    single_data = target_2631.loc[index]
  # is_data_valid = (single_data['latitude'] < 38.85) & (single_data['latitude'] > 38.099) & (single_data['longitude'] < -122.35) & (single_data['longitude'] > -123.55)
  
    if data_id != start_id:
    # print("end index: %d, caid: %s" %(index - 1, target_2631.loc[index - 1]['caid']))
        res+=')'
        df_new.loc[count] = {'index': count, 'caid': target_2631.loc[index - 1]['caid'], 'start_time' : start_time,
                             'end_time' : target_2631.loc[index - 1]['la_datetime'],'count' : (index - start_index),
                             'is_process' : False, 'is_success' : False, 'res' : res, 'time' : time_array,
                             'distance' : distance_array, 'speed' : speed_array, 'direction': direction_array }
        is_first = True
        res = 'LINESTRING('
        count+=1
        start_index = index
        start_id = data_id
        start_time = target_2631.loc[index]['la_datetime']
        time_array = []
        distance_array = []
        speed_array = []
        direction_array = []
    time = single_data['la_datetime']
    lat = float(single_data['latitude'])
    lon = float(single_data['longitude'])
    if is_first:
        is_first = False
        res+= (str(lon) + ' ' + str(lat))
    else:
        res+= (',' + str(lon) + ' ' + str(lat))
        t = (time - prev_time) / pd.Timedelta('1 second')
        time_array.append(t)
        loc1=(prev_lat, prev_lon)
        loc2=(lat, lon)
        x, d = calculate_bearing(prev_lat, prev_lon, lat, lon)
        distance_array.append(x)
        speed_array.append(x / t * 2.23693629)
        direction_array.append(d)
    prev_time = time
    prev_lat = lat
    prev_lon = lon
    
    

start index: 0, caid: 0000dec3ca31d5b1b07771650fb6ee077059a5a579f0e35c30d5d68a0008feb6


In [82]:
np.array(df_new.loc[4]['speed']).mean()

13.670156849504076

In [85]:
df_new.to_csv('data/GPS.nosync/Sonoma-2631-new-points/out3.csv')

In [33]:
df = pd.read_csv('data/GPS.nosync/Sonoma-2631-new-points/out2.csv')

In [48]:
time = df['start_time'][4]
print(time)
pd.Timestamp(time)

2019-10-25 17:03:02-07:00


Timestamp('2019-10-25 17:03:02-0700', tz='pytz.FixedOffset(-420)')